In [69]:
!pip install transformers

In [0]:
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
#Also based on the following tutorials
#https://mccormickml.com/2019/07/22/BERT-fine-tuning/

In [0]:
import tensorflow as tf
import torch

import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


from transformers import BertTokenizer as bertTokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler

from sklearn.svm import SVC
from sklearn import preprocessing

from transformers import BertForSequenceClassification as bfsc,AdamW,BertConfig
from torch.utils.data import Dataset
from transformers import get_linear_schedule_with_warmup


from __future__ import absolute_import, division, print_function

import glob
import logging
import os
import random
import json

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import random
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm_notebook, trange


from transformers import AdamW#, WarmupLinearSchedule

#from utils import (convert_examples_to_features,
                        #output_modes, processors)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [0]:
gpuname=""
device=""
y=""
preprocessedTweets=""
ids_of_sentence=[]
ids_of_sentence_words=[]


In [117]:


gpuname=tf.test.gpu_device_name()
if gpuname=='/device:GPU:0':
  print('Found GPU at :{}'.format(gpuname))
else:
  gpuname=""
if torch.cuda.is_available():
  device=torch.device("cuda")
  n_gpu=torch.cuda.device_count()
  print("The device name is %s"%torch.cuda.get_device_name(0))
else:
  print("No GPU available using only CPU instead")
  device=torch.device("cpu")




Found GPU at :/device:GPU:0
The device name is Tesla P100-PCIE-16GB


In [118]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
##NOT FOR SYSTEMS
#!unzip -P ****** -qq '/content/drive/My Drive/EnglishData/task_a_distant.zip' 

In [0]:
#!unzip -P ****** -qq '/content/drive/My Drive/EnglishData/task_b_distant.zip' 

In [77]:
'''def convertToFloat(val):
    if not val:
        return 0    
    try:
        return np.float64(val)
    except:        
        return np.float64(0)


headers=['id','text','average','std']
englishdata1 = pd.read_csv("task_b_distant.tsv", delimiter='\t',names=headers,low_memory=False,converters={"average":convertToFloat,"std":convertToFloat})
englishdata1=englishdata1'''

'def convertToFloat(val):\n    if not val:\n        return 0    \n    try:\n        return np.float64(val)\n    except:        \n        return np.float64(0)\n\n\nheaders=[\'id\',\'text\',\'average\',\'std\']\nenglishdata1 = pd.read_csv("task_b_distant.tsv", delimiter=\'\t\',names=headers,low_memory=False,converters={"average":convertToFloat,"std":convertToFloat})\nenglishdata1=englishdata1'

In [78]:
'''def convertToFloat(val):
    if not val:
        return 0    
    try:
        return np.float64(val)
    except:        
        return np.float64(0)


headers=['id','text','average','std']
englishdata2 = pd.read_csv("task_a_distant.tsv", delimiter='\t',names=headers,low_memory=False,converters={"average":convertToFloat,"std":convertToFloat})
englishdata2=englishdata2'''

'def convertToFloat(val):\n    if not val:\n        return 0    \n    try:\n        return np.float64(val)\n    except:        \n        return np.float64(0)\n\n\nheaders=[\'id\',\'text\',\'average\',\'std\']\nenglishdata2 = pd.read_csv("task_a_distant.tsv", delimiter=\'\t\',names=headers,low_memory=False,converters={"average":convertToFloat,"std":convertToFloat})\nenglishdata2=englishdata2'

In [0]:
#len(englishdata1)

In [0]:
#len(englishdata2)

In [0]:

#GET THE DATA FROM THE PANDAS FRAME
headers=['id','tweet','subtask_a','subtask_b','subtask_c']
englishdata = pd.read_csv("olid-training-v1.0.tsv", delimiter='\t',names=headers,low_memory=False)
englishdata=englishdata[['id','tweet','subtask_a']]
englishdata=englishdata[1:]

In [83]:
'''# conversion to float taken from https://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options
def convertToFloat(val):
    if not val:
        return 0    
    try:
        return np.float64(val)
    except:        
        return np.float64(0)


#GET THE DATA FROM THE PANDAS FRAME
headers=['id','text','average','std']
englishdata = pd.read_csv("task_a_distant.tsv", delimiter='\t',names=headers,low_memory=False,converters={"average":convertToFloat,"std":convertToFloat})
englishdata=englishdata[:100]
'''

'# conversion to float taken from https://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options\ndef convertToFloat(val):\n    if not val:\n        return 0    \n    try:\n        return np.float64(val)\n    except:        \n        return np.float64(0)\n\n\n#GET THE DATA FROM THE PANDAS FRAME\nheaders=[\'id\',\'text\',\'average\',\'std\']\nenglishdata = pd.read_csv("task_a_distant.tsv", delimiter=\'\t\',names=headers,low_memory=False,converters={"average":convertToFloat,"std":convertToFloat})\nenglishdata=englishdata[:100]\n'

In [144]:
len(englishdata)

13240

In [85]:
englishdata.head()

,id,tweet,subtask_a
1,86426,@USER She should ask a few native Americans wh...,OFF
2,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF
3,16820,Amazon is investigating Chinese employees who ...,NOT
4,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF
5,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT


In [120]:
'''englishtrain,englishtest= train_test_split(englishdata, test_size=0.2, random_state=42)
export_csv = englishtrain.to_csv ('/content/drive/My Drive/EnglishData/olidlearn/TrainFileEnglish.tsv', index = None, header=True)
print (englishtrain.head())
#englishtest,englishpredict= train_test_split(englishtemp, test_size=0.5, random_state=42)
export_csv = englishtest.to_csv ('/content/drive/My Drive/EnglishData/olidlearn/TestFileEnglish.tsv', index = None, header=True)
print (englishtest.head())'''
#export_csv = englishpredict.to_csv ('/content/drive/My Drive/EnglishData/predictFileEnglish.csv', index = None, header=True)
#print (englishpredict.head())

          id                                              tweet subtask_a
11461  51336                        @USER She is not leaving BB       NOT
7946   78922  @USER @USER Ford and the conservatives hates t...       NOT
9186   10317                         @USER God is good to us 🙏👍       NOT
1057   14809  @USER woman accusing Supreme Court nominee #Ka...       NOT
1147   34120  @USER CORRECTION: The Liberals won a false-maj...       NOT
          id                                              tweet subtask_a
12388  27650  @USER @USER Why is John Kerry running his mout...       NOT
386    52965        @USER @USER Gun control anyone? #DisarmHate       NOT
4897   87438                      @USER What manga you reading?       NOT
3972   67140  @USER You are right. Victoria is on the revers...       NOT
10395  43102  @USER @USER Thank you @USER . America is respe...       NOT


In [87]:
'''ids_of_sentence=[]
ids_of_sentence_words=[]
attention_masks=[]

def giveIds(sentence):
  maxlength=0
  tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
  for t in sentence:
      tokenized_sentence_id=tokenizer.encode(t,add_special_tokens=True)
      if(maxlength<len(tokenized_sentence_id)):
          maxlength=len(tokenized_sentence_id)
  return maxlength'''

"ids_of_sentence=[]\nids_of_sentence_words=[]\nattention_masks=[]\n\ndef giveIds(sentence):\n  maxlength=0\n  tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)\n  for t in sentence:\n      tokenized_sentence_id=tokenizer.encode(t,add_special_tokens=True)\n      if(maxlength<len(tokenized_sentence_id)):\n          maxlength=len(tokenized_sentence_id)\n  return maxlength"

In [0]:
#max_length=

In [89]:
set(y)

set()

In [0]:
'''
from sklearn.svm import SVC
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

# conversion to float taken from https://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options
def convertToFloat(val):
    if not val:
        return 0    
    try:
        return np.float64(val)
    except:        
        return np.float64(0)

def readDataTrain():
  headers=['id','tweet','subtask_a']
  edata = pd.read_csv("/content/drive/My Drive/EnglishData/olidlearn/TrainFileEnglish.tsv", delimiter=',',names=headers,low_memory=False)
  edata=edata[1:]
  dfnumpy=edata.to_numpy();
  X=dfnumpy[:, 1].reshape(-1, 1)
  y=dfnumpy[:, 2].reshape(-1, 1)
  y1=le.fit_transform(y.flatten())
  print(set(y1))
  print(le.classes_)
  return X,y1

def readDataTest():
  headers=['id','tweet','subtask_a']
  edata = pd.read_csv("/content/drive/My Drive/EnglishData/olidlearn/TrainFileEnglish.tsv", delimiter=',',names=headers,low_memory=False)
  edata=edata[1:]
  dfnumpy=edata.to_numpy();
  X=dfnumpy[:, 1].reshape(-1, 1)
  y=dfnumpy[:, 2].reshape(-1, 1)
  y1=le.fit_transform(y.flatten())

  return X,y1'''

In [0]:
dfnumpy=englishdata.to_numpy()
X=dfnumpy[:, 1].reshape(-1, 1)
y=dfnumpy[:, 2].reshape(-1, 1)

In [158]:
y[0]

array(['NOT'], dtype=object)

In [0]:
preprocessedTweets=X[:,0]

In [0]:
from sklearn.utils import shuffle
preprocessedTweets,y=shuffle(preprocessedTweets,y)

In [0]:
x_train,x_test,y_train,y_test= train_test_split(preprocessedTweets,y, test_size=0.2, random_state=42)


In [155]:
x_train

array(['@USER @USER @USER @USER I never said no one died in Sa Juan..I said she is only in charge of one town out of 78...don’t get me wrong many people to blame...in the case of Ricardo Rosello he is terrified of Trump and wants to please him at all cause.  Im sure Roselló and trump knew the count was higher',
       '@USER @USER @USER @USER @USER Now they have really sunk to a new low !! Hope the show BOMBS ! The entire thing a set up by libs just like everything else they are doing !! Roseanne 🌹; was the show !! MAGA 🇺🇸',
       '@USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER @USER Same here. You got a follow. Great talk on my way home from work. Time to be the vampire and sleep during the day',
       ...,
       '@USER He is like a cheap plastic version of a real president.',
       '@USER @USER @USER Totally. Just shown herself up to be a spoilt narcissist. The comment to the umpire that he will never be at her matches for as long as he lives? Just who do

In [122]:
'''x_train,y_train=readDataTrain()
x_test,y_test=readDataTest()
'''

{0, 1}
['NOT' 'OFF']


In [157]:
print(preprocessedTweets[0:5])

['@USER @USER @USER @USER Becca. She is too much work.'
 "@USER Disagree; he was chosen explicitly to make the President immune to the rule of law. All of his heinous beliefs on women's rights and gun control are just the frosting to make the GOP confirm him."
 '@USER That shit weird! Lol'
 '#WebberForCongress #NJ11 #MAGA we cannot allow #Liberals to #Impeach #POTUS #Trump URL'
 '@USER I’m sorry you are going through this.']


In [159]:
from sklearn.svm import SVC
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

print(y_train.shape)
print(y_test.shape)
#print(y_predict.shape)
yTrain=le.fit_transform(y_train.flatten())
print(yTrain.shape)
print(le.classes_)
yTest=le.fit_transform(y_test.flatten())
print(le.classes_)

(10592, 1)
(2648, 1)
(10592,)
['NOT' 'OFF']
['NOT' 'OFF']


In [93]:
set(y_test)

{0, 1}

In [94]:
###IMPORTANT
'''xtest,x_test_mask=giveIds(x_test.flatten(),y_test)
x_test_pytorch=torch.tensor(xtest)
y_test_pytorch=torch.tensor(y_test)
x_test_mask_pytorch=torch.tensor(x_test_mask)
tedata=TensorDataset(x_test_pytorch,x_test_mask_pytorch,y_test_pytorch)
tesampler=RandomSampler(tedata)
bsize=64
tedataloader=DataLoader(tedata,sampler=tesampler,batch_size=bsize)
print(len(xtest))
print(len(y_test))
len(x_train)
'''

'xtest,x_test_mask=giveIds(x_test.flatten(),y_test)\nx_test_pytorch=torch.tensor(xtest)\ny_test_pytorch=torch.tensor(y_test)\nx_test_mask_pytorch=torch.tensor(x_test_mask)\ntedata=TensorDataset(x_test_pytorch,x_test_mask_pytorch,y_test_pytorch)\ntesampler=RandomSampler(tedata)\nbsize=64\ntedataloader=DataLoader(tedata,sampler=tesampler,batch_size=bsize)\nprint(len(xtest))\nprint(len(y_test))\nlen(x_train)\n'

In [281]:
from transformers import RobertaForSequenceClassification as bfsc,AdamW,RobertaConfig
from transformers import RobertaModel, RobertaTokenizer
from transformers import RobertaForSequenceClassification, RobertaConfig

config = RobertaConfig.from_pretrained('roberta-base',output_attentions=False,output_hidden_states=False,num_labels=2)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification(config)
model.cuda()

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.a7ab0e5de2d8321d6d6a15b199110f2c99be72976b7d151423cb8d8c261a13b6
INFO:transformers.configuration_utils:Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_head

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [282]:

torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/EnglishData/robertaenglish.pth.tar')
checkpoint = torch.load('/content/drive/My Drive/EnglishData/robertaenglish.pth.tar')
model.load_state_dict(checkpoint['state_dict'])
  

<All keys matched successfully>

In [0]:
params=list(model.named_parameters())

In [0]:
no_decay = ["bias", "beta","LayerNorm.weight","gamma"]
optimizer_grouped_parameters = [
{
"params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
"weight_decay": 0.01,
},
{"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]

In [0]:
optimizer=AdamW(model.parameters(),lr=3e-5,eps=1e-8)

In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

def calculateF1Score(predictions,labels):
  #rowwise return the index of the max element ie 0 or 1 depending on the maximum value returned
  predictionArgmax=np.argmax(predictions,axis=1).flatten()
  labelsFlattend=labels.flatten()
  print("Predictions Argmax",predictionArgmax)
  print("labels Flattened",labelsFlattend)   
  return f1_score(labelsFlattend, predictionArgmax, average='macro'),accuracy_score(labelsFlattend, predictionArgmax)



In [288]:
###IMPORTANT
'''
xtest,x_test_mask=giveIds(x_test.flatten(),y_test)
x_test_pytorch=torch.tensor(xtest)
y_test_pytorch=torch.tensor(y_test)
x_test_mask_pytorch=torch.tensor(x_test_mask)
tedata=TensorDataset(x_test_pytorch,x_test_mask_pytorch,y_test_pytorch)
tesampler=RandomSampler(tedata)
bsize=64
tedataloader=DataLoader(tedata,sampler=tesampler,batch_size=bsize)
print(len(xtest))
print(len(y_test))
len(x_train)
'''

'\nxtest,x_test_mask=giveIds(x_test.flatten(),y_test)\nx_test_pytorch=torch.tensor(xtest)\ny_test_pytorch=torch.tensor(y_test)\nx_test_mask_pytorch=torch.tensor(x_test_mask)\ntedata=TensorDataset(x_test_pytorch,x_test_mask_pytorch,y_test_pytorch)\ntesampler=RandomSampler(tedata)\nbsize=64\ntedataloader=DataLoader(tedata,sampler=tesampler,batch_size=bsize)\nprint(len(xtest))\nprint(len(y_test))\nlen(x_train)\n'

In [0]:
#z=len(x_test)

In [0]:
#creating a dataset inspired from 
#https://towardsdatascience.com/bert-classifier-just-another-pytorch-model-881b3cf05784





In [0]:
#xytest[1]

In [0]:
#the customdataset section has been inspired from 
#https://github.com/sugi-chan/custom_bert_pipeline/blob/master/bert_pipeline.ipynb  

In [0]:
MAXLENGTH=64

In [293]:
class EnglishTrainDataset(Dataset):
    def __init__(self,xytrain):
        self.xytrain = xytrain
        self.maxlength=MAXLENGTH
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(self.xytrain[0][index])
        if len(tokenized_review) > self.maxlength:
            #print(tokenized_review)
            tokenized_review = tokenized_review[:self.maxlength]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        #attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_train_pytorch = torch.tensor(ids_of_sentence_word)
        y_train_pytorch=torch.tensor(self.xytrain[1][index])
        
        return x_train_pytorch,y_train_pytorch
        #return [1,2,3]
        
    def __len__(self):
        return len(self.xytrain[0])
 

'''
z=0;
for batch_idx, data in enumerate(tdataloader): 
  if z==100:
    break;
  z=z+1;'''

'\nz=0;\nfor batch_idx, data in enumerate(tdataloader): \n  if z==100:\n    break;\n  z=z+1;'

In [0]:
class EnglishTestDataset(Dataset):
    def __init__(self,xytest):
        self.xytest = xytest
        self.maxlength=MAXLENGTH
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(self.xytest[0][index])
        if len(tokenized_review) > self.maxlength:
            #print(tokenized_review)
            tokenized_review = tokenized_review[:self.maxlength]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        #assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        #attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_test_pytorch = torch.tensor(ids_of_sentence_word)
        y_test_pytorch=torch.tensor(self.xytest[1][index])
        #x_test_mask_pytorch=torch.tensor(attention_mask)
        
        return x_test_pytorch,y_test_pytorch
        #return [1,2,3]
        
    def __len__(self):
        return len(self.xytest[0])



In [0]:
#xytrain=[x_train[:8000],y_train[:8000]]\
#MAXLENGTH=giveIds(x_train[0])
xytrain=[x_train,yTrain]
#tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
tdataset = EnglishTrainDataset(xytrain)
tsampler=RandomSampler(tdataset)
tdataloader = DataLoader(tdataset, batch_size=32, num_workers=1, shuffle=False,sampler=tsampler)
      

In [296]:
#tokenized_review = tokenizer.tokenize(str(xytest[0][0].flatten()))
 #y_test_pytorch=torch.tensor(y_test[0])
 #y_test[0]
 xytrain[0][1]

'@USER @USER @USER @USER @USER Now they have really sunk to a new low !! Hope the show BOMBS ! The entire thing a set up by libs just like everything else they are doing !! Roseanne 🌹; was the show !! MAGA 🇺🇸'

In [0]:
#tokenized_review

In [0]:

#xytest=[x_test[:8000],y_test[:8000]]
xytest=[x_test,yTest]
#tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
tedataset = EnglishTestDataset(xytest) 
tesampler=RandomSampler(tedataset)
tedataloader = DataLoader(tedataset, batch_size=32, num_workers=1, shuffle=False,sampler=tesampler)
#trainData(tdataloader,tedataloader)

In [0]:

epochs=4
total_steps=len(tdataloader)*epochs
sch=get_linear_schedule_with_warmup(optimizer,
                                    num_warmup_steps=0,num_training_steps=total_steps)



In [300]:
import random
import time 

def set_seed(seed,ngpu):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if ngpu > 0:
        torch.cuda.manual_seed_all(seed)
      
set_seed(42,torch.cuda.device_count())
#remove later

epochs=4
lossList=[]
max_grad_norm=1.0
for e in range(0, epochs):
    print("Start Epoch Number",(e + 1))
    print("Start Training")
    

    #Amount of time taken for training
    t1 = time.time()
    tr_loss, logging_loss = 0.0, 0.0
    model.train()
    tsteps=0
    for step, batch in enumerate(tdataloader):
        if step % 50 == 0 and not step == 0:
            print("Batch Completed  {:,}  of  {:,}.    Elapsed time is  {}".format(step, len(tdataloader),time.time() - t1))
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "labels": batch[1]}
        model.zero_grad()     
        outputs = model(inputs["input_ids"],token_type_ids=None, labels=inputs["labels"])
        #print(type(outputs))
        loss, prediction_scores = outputs[:2]
        #print(loss)
        loss.backward()
        tr_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        tsteps+=1
        optimizer.step()
        sch.step()
    a_tr_loss = tr_loss /(tsteps)               
    lossList.append(a_tr_loss)
    print(" The training loss incured is  {0:.3f}".format(a_tr_loss))
    t2=time.time()
    print("  Training one epoch time taken",t2-t1)
    print(" Validation starts here ")
    t1 = time.time()
    model.eval()
    eval_loss = 0
    nb_eval_steps = 0
    eval_f1=0
    eval_acc=0
    
    for batch_idx, data in enumerate(tedataloader):
        
        batch = tuple(t.to(device) for t in data)            
        inputs = {"input_ids": batch[0], "labels": batch[1]}
        with torch.no_grad():        
           outputs = model(inputs["input_ids"],token_type_ids=None)
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = (inputs["labels"]).to('cpu').numpy()
        tmpf1score,tmpaccscore = calculateF1Score(logits, label_ids)
        eval_f1 = eval_f1+tmpf1score
        eval_acc=eval_acc+tmpaccscore
        nb_eval_steps += 1
        #print(" TEMP F1 score: {0:.3f}".format(tmpf1score))
        #print("TEMP  Accuracy score: {0:.3f}".format(tmpaccscore))
    #torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/EnglishData/robertaenglish.pth.tar')
    print("  F1 score: {0:.3f}".format(eval_f1/nb_eval_steps))
    print("  Accuracy score: {0:.3f}".format(eval_acc/nb_eval_steps))
    t2=time.time()
    print("  Validating one epoch time taken ",t2-t1)
      
    
print("ALL DONE!!!")

Start Epoch Number 1
Start Training
Batch Completed  50  of  331.    Elapsed time is  11.876701593399048
Batch Completed  100  of  331.    Elapsed time is  23.65893268585205
Batch Completed  150  of  331.    Elapsed time is  35.394779920578
Batch Completed  200  of  331.    Elapsed time is  47.17995762825012
Batch Completed  250  of  331.    Elapsed time is  59.05367064476013
Batch Completed  300  of  331.    Elapsed time is  70.75678062438965
 The training loss incured is  0.652
  Training one epoch time taken 78.07373690605164
 Validation starts here 
Predictions Argmax [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
labels Flattened [1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0]
Predictions Argmax [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
labels Flattened [0 0 0 1 0 1 0 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0]
Predictions Argmax [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
labels Flattened [1

In [301]:
print(type(outputs))

<class 'tuple'>


In [302]:
print(outputs[1])

IndexError: ignored

In [0]:
'''nlists=2
last=16000
size_of_itertrain=8000//nlists
size_of_itertest=8000//nlists
#print(size_of_itertrain)
for  i in range(nlists+1):
      
      end=(i+1)*size_of_itertrain
      if(last<end):
        end=last
      xytrain=[x_train[i*size_of_itertrain:end],y_train[i*size_of_itertrain:end]]
      tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
      tdataset = EnglishTrainDataset(xytrain)
      tsampler=RandomSampler(tdataset)
      tdataloader = DataLoader(tdataset, batch_size=32, num_workers=1, shuffle=False,sampler=tsampler)
      
      epochs=2
      total_steps=len(tdataloader)*epochs
      sch=get_linear_schedule_with_warmup(optimizer,
                                    num_warmup_steps=0,num_training_steps=total_steps)

      end=(i+1)*size_of_itertest
      if(last<end):
        end=last 
    
      xytest=[x_test[i*size_of_itertest:end],y_test[i*size_of_itertest:end]]
      tokenizer=bertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
      tedataset = EnglishTestDataset(xytest)  
      tesampler=RandomSampler(tedataset)
      tedataloader = DataLoader(tedataset, batch_size=32, num_workers=1, shuffle=False,sampler=tesampler)

      trainData(tdataloader,tedataloader)

      
y_test_pytorch=torch.tensor(xytest[1][index])'''

In [0]:
#y_test_pytorch=torch.tensor(xytest[1][40])

In [0]:
#len(xytest[1])

In [0]:
#len(xytest[0])

In [0]:
#len(xytrain[1])

In [0]:

MAXLENGTH

In [0]:

print("MAXLENGTH",MAXLENGTH)